In [10]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import mlflow

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db

# set the tracking uri
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# set experiment
mlflow.set_experiment("nyc-taxi-experiment")

2024/06/14 08:16:38 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/deployment/mlruns/1', creation_time=1718345798690, experiment_id='1', last_update_time=1718345798690, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
  if filename.endswith('.csv'):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  elif filename.endswith('.parquet'):
    df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
    
  return df

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
def prepare_features(df, dv=None):
  # merge location columns
  df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

  # define predictor and target variables
  categorical = ['PU_DO'] #'PULocationID', 'DOLocationID'
  numerical = ['trip_distance']
  target = 'duration'

  # dataframe to dictionaries for vectorizing
  data_dicts = df[categorical + numerical].to_dict(orient='records')

  if dv:
    X = dv.transform(data_dicts)
  else:
    dv = DictVectorizer()
    X = dv.fit_transform(data_dicts)

  y = df[target].values

  return X, y, dv

In [8]:
X_train, y_train, dv = prepare_features(df=df_train)
X_val, y_val, _ = prepare_features(df=df_val, dv=dv)

In [9]:
with mlflow.start_run():
  mlflow.set_tag('Developer', 'Armand Winant')

  # log the datasets used for training and validation
  mlflow.log_param("training-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
  mlflow.log_param("validation-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

  # set and log model parameter
  alpha = .01
  mlflow.log_param("alpha", alpha)

  # create and train model
  lr = Lasso(alpha=alpha)
  lr.fit(X_train, y_train)

  # compute and log validation error
  y_pred = lr.predict(X=X_val)
  rmse = root_mean_squared_error(y_pred, y_val)
  mlflow.log_metric("rmse", rmse)

### HYPERPARAMETER TUNING (GRID SEARCH)

In [11]:
train = xgb.DMatrix(data=X_train, label=y_train)
valid = xgb.DMatrix(data=X_val, label=y_val)

In [12]:
# HYPERPARAMETER TUNING (GRID SEARCH)
def objective(params):
  with mlflow.start_run():
    # log the model and params
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)

    # train the model
    booster = xgb.train(
      params = params,
      dtrain = train,
      num_boost_round = 10,
      evals = [(valid, "validation")],
      early_stopping_rounds = 5
    )

    # generate validation preds and evaluate error
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
  
  return { 'loss': rmse, 'status': STATUS_OK }

In [15]:
search_space = {
  'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
  'objective': 'reg:linear',
  'seed': 42
}

best_result = fmin(
  fn=objective,
  space=search_space,
  algo=tpe.suggest,
  max_evals=50,
  trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:47:12] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.49105                           
[1]	validation-rmse:6.70234                           
[2]	validation-rmse:6.54988                           
[3]	validation-rmse:6.50555                           
[4]	validation-rmse:6.48368                           
[5]	validation-rmse:6.46882                           
[6]	validation-rmse:6.45914                           
[7]	validation-rmse:6.45479                           
[8]	validation-rmse:6.44882                           
[9]	validation-rmse:6.44291                           
[0]	validation-rmse:8.32201                                                  
  2%|▏         | 1/50 [00:03<02:39,  3.25s/trial, best loss: 6.4429131051199]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:47:15] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.16137                                                  
[2]	validation-rmse:6.83951                                                  
[3]	validation-rmse:6.73629                                                  
[4]	validation-rmse:6.69551                                                  
[5]	validation-rmse:6.67789                                                  
[6]	validation-rmse:6.66314                                                  
[7]	validation-rmse:6.65960                                                  
[8]	validation-rmse:6.65589                                                  
[9]	validation-rmse:6.65142                                                  
  4%|▍         | 2/50 [00:04<01:28,  1.84s/trial, best loss: 6.4429131051199]

...

[0]	validation-rmse:11.11115                                                    
[1]	validation-rmse:10.21081                                                    
[2]	validation-rmse:9.47685                                                     
[3]	validation-rmse:8.88802                                                     
[4]	validation-rmse:8.41300                                                     
[5]	validation-rmse:8.03373                                                     
[6]	validation-rmse:7.73414                                                     
[7]	validation-rmse:7.50219                                                     
[8]	validation-rmse:7.31013                                                     
[9]	validation-rmse:7.16570                                                     
[0]	validation-rmse:8.42893                                                     
 96%|█████████▌| 48/50 [05:01<00:16,  8.30s/trial, best loss: 6.423900650655789]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:52:12] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.23589                                                     
[2]	validation-rmse:6.88818                                                     
[3]	validation-rmse:6.78706                                                     
[4]	validation-rmse:6.73037                                                     
[5]	validation-rmse:6.70956                                                     
[6]	validation-rmse:6.70335                                                     
[7]	validation-rmse:6.70151                                                     
[8]	validation-rmse:6.69548                                                     
[9]	validation-rmse:6.69295                                                     
 98%|█████████▊| 49/50 [05:01<00:06,  6.12s/trial, best loss: 6.423900650655789]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [08:52:13] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.30772                                                     
[1]	validation-rmse:6.70467                                                     
[2]	validation-rmse:6.59407                                                     
[3]	validation-rmse:6.55637                                                     
[4]	validation-rmse:6.53865                                                     
[5]	validation-rmse:6.52345                                                     
[6]	validation-rmse:6.51332                                                     
[7]	validation-rmse:6.50944                                                     
[8]	validation-rmse:6.50200                                                     
[9]	validation-rmse:6.49970                                                     
100%|██████████| 50/50 [05:04<00:00,  6.10s/trial, best loss: 6.423900650655789]
